<a href="https://colab.research.google.com/github/NbtKmy/gc_workshops/blob/main/folium_TEI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Karten erstellen mit Pyhton

Hier bringen wir unterschiedliche Daten (z.B. die mit [TEI](https://tei-c.org/) codierten Texte) auf eine geographische Karte.


**Arbeiten in Google Colab**

In Google Colab kann man entweder Code-Zell oder Text-Zell hinzufügen und schreiben.
In einem Code-Zell wird i.d.R. Python geschrieben. Man kann aber auch Ipython und Shell Befehle ausführen, indem man das "!"- oder "%"-Zeichen vor dem Befehl voran stellt. Näheres [hier](https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/01.05-IPython-And-Shell-Commands.ipynb)

In einem Text-Zell kann man Texte in Markdown-Format verfassen.

**Shortcutkeys in Colab**

[Hier](https://docs.google.com/spreadsheets/d/1diiQZxq_G6XiPmSiJBWn8comGpiH-sEyYHjXWkGmrkg/edit?usp=sharing) findet man die Shortcutkeys in Google Colab. Sie könnten nützlich sein...

In [ ]:
! pip install -U folium

In [ ]:
import folium
import folium.plugins as plugin

tile_name = 'cartodbdark_matter'


testmap = folium.Map(
    location=[47.37174, 8.54226],
    zoom_start=12,
    tiles=tile_name
)



## TEI-Textdaten auf die Karte zeichnen



### TEI-Daten von correspSearch

Die TEI-Daten holen wir zuerst von [correspSearch API](https://correspsearch.net/de/api.htm).

API-Abfrage
```
https://correspsearch.net/api/v1.2/tei-xml.xql?correspondent=http://viaf.org/viaf/40175198&startdate=1840-01-01&enddate=1840-06-01
```

Damit wir den Server von correspSearch nicht zu sehr belasten, habe ich die Kopie der Ergebnis-Liste als Gist erstellt.
```
https://gist.githubusercontent.com/NbtKmy/4b120870a354bae415f5f8aa9c43a97c/raw/f359c553a6ee89a73b7e8de403e3a0cd9487a5e0/escherCorresp_test.xml
```
Die Daten, die durch API geholt sind, sind unter den Bedingungen der **Creative-Commons-Lizenz CC-BY 4.0** zu verwenden!!

Zuerst betrachten wir die TEI-Datei:

In [ ]:
!curl -O https://gist.githubusercontent.com/NbtKmy/4b120870a354bae415f5f8aa9c43a97c/raw/f359c553a6ee89a73b7e8de403e3a0cd9487a5e0/escherCorresp_test.xml
!cat escherCorresp_test.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5645  100  5645    0     0  25200      0 --:--:-- --:--:-- --:--:-- 25200
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>correspSearch API - Correspondence Descriptions from divers sources</title>
                <respStmt>
                    <resp>Data provider of the correspondence descriptions</resp>
                    <name>
                    Jung, Joseph (Hrsg.) (2015), Digitale Briefedition Alfred Escher. Version: Juli 2015, Zürich: Alfred Escher-Stiftung
                </name>
                </respStmt>
            </titleStmt>
            <publicationStmt>
                <publisher>
                    <ref target="http://www.bbaw.de">Berlin-Brandenburgische Akademie der Wissenschaften</ref>
                </publisher>
               

### GeoNames?


In der TEI-Datei sind keine Koordinaten zu finden.
Stattdessen stehen die URIs für **GeoNames-API** wie "http://www.geonames.org/2660594" für Glarus.
Um aus dieser URI die Koordinanten zu holen, verwenden wir eine weitere API.

Die API-Abfrage sieht so aus:
https://sws.geonames.org/2660594/about.rdf

Sie wirft ein XML-File zurück

In [ ]:
!curl -o test.xml https://sws.geonames.org/2660594/about.rdf
!cat test.xml

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4612    0  4612    0     0   8669      0 --:--:-- --:--:-- --:--:--  8669
<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<rdf:RDF xmlns:cc="http://creativecommons.org/ns#" xmlns:dcterms="http://purl.org/dc/terms/" xmlns:foaf="http://xmlns.com/foaf/0.1/" xmlns:gn="http://www.geonames.org/ontology#" xmlns:owl="http://www.w3.org/2002/07/owl#" xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#" xmlns:rdfs="http://www.w3.org/2000/01/rdf-schema#" xmlns:wgs84_pos="http://www.w3.org/2003/01/geo/wgs84_pos#">
    <gn:Feature rdf:about="https://sws.geonames.org/2660594/">
        <rdfs:isDefinedBy rdf:resource="https://sws.geonames.org/2660594/about.rdf"/>
        <gn:name>Glarus</gn:name>
        <gn:alternateName xml:lang="ko">글라루스</gn:alternateName>
        <gn:alternateName xml:lang="ia">Canton Glarus</gn:alternateName>


Unter den Tags wgs84_pos:lat und wgs84_pos:long findet man die GEO-Daten.

### Daten aus TEI exrahieren und auf die Karte darstellen

Um aus der TEI-Datei die uns interessanten Inforamtion herauszuholen, benutzen wir Python-Library [BeautifulSoup](https://beautiful-soup-4.readthedocs.io/en/latest/). Die Library erleichtert uns die Daten aus HTML- und XML-Dokument zu holen. Weil wir hier XML behandeln, wird hier auch noch [lxml](https://pypi.org/project/lxml/), XML-Parser, installiert.

In [ ]:
!pip install bs4
!pip install lxml

In [ ]:
from bs4 import BeautifulSoup
import re
import requests
import folium
import folium.plugins as plugin


data = ''
with open('escherCorresp_test.xml') as f:
    data = f.read()


soup = BeautifulSoup(data, 'lxml-xml')

# Link zu 'Escher Briefedition' sammeln
all_correspDescs = soup.find_all('correspDesc')
links_list = []
for correspDesc in all_correspDescs:
    link = correspDesc['ref']
    links_list.append(link)


# Sender-Information sammeln
all_senders = soup.find_all(type='sent')
senders_list = []
for sender in all_senders:

    sender_name = sender.select('persName')[0]
    name = sender_name.get_text()
    if len(sender.select('placeName')) > 0:
        sender_ort = sender.select('placeName')[0]
        ort_link = sender_ort['ref']
        ort_id = re.sub(r'\D', '', ort_link)

        # Von GeoNames die Koordinanten-information holen
        req_link = 'https://sws.geonames.org/' + ort_id + '/about.rdf'
        req = requests.get(req_link)
        geo_data = req.text
        geo_soup = BeautifulSoup(geo_data, 'lxml-xml')
        lat_string = geo_soup.find('wgs84_pos:lat').get_text()
        long_string = geo_soup.find('wgs84_pos:long').get_text()
        ort = [ float(lat_string), float(long_string) ]

    else:
        # Wenn Escher als Sender steht, gibt es keine Ortsangabe. Deshalb wird ein provisorischer Ort 'Zürich' zugewiesen.
        if name == 'Alfred Escher':
            ort = [47.36667, 8.55]
            ortsname = 'Zürich'

    d = sender.find('date')
    date = d['when']
    sender = { 'sender_name': name, 'geo': ort, 'date': date }
    senders_list.append(sender)

# Empfänger-Information sammeln
all_receivers = soup.find_all(type='received')
receivers_list = []
for receiver in all_receivers:
    receiver_name = receiver.select('persName')
    if len(receiver_name) > 0:
        name = receiver_name[0].get_text()

        # Wenn der Empfänger 'Alfred Escher' ist, ist der Ort Zürich (47.36667, 8.55) nach GeoNames
        if name == 'Alfred Escher':
            ort = [47.36667, 8.55]
            receiver = { 'receiver_name': name, 'geo': ort }
            receivers_list.append(receiver)
        elif name == 'Arnold Otto Aepli':
            for x in senders_list:
                if 'Arnold Otto Aepli' in x.values():
                    ort = x.get('geo')
                    break
            receiver = { 'receiver_name': name, 'geo': ort}
            receivers_list.append(receiver)
    else:
        # In diesem Beispiel haben wir nur Centralausschuss des Zofingervereins. Dafür wird ein provisorischer Ort 'Zofingen' zugewiesen
        receiver_name = receiver.select('orgName')[0]
        name = receiver_name.get_text()
        ort = [47.28845, 7.94497]
        receiver = { 'receiver_name': name, 'geo': ort}
        receivers_list.append(receiver)



# In links_list, senders_list und receivers_list sind die Daten für AntPath drin
# Karte Zeichnen

tile_name = 'cartodbdark_matter'


testmap = folium.Map(
    location=[47.37174, 8.54226],
    zoom_start=12,
    tiles=tile_name
)

for i in range(len(links_list)):
    path = [senders_list[i]['geo'], receivers_list[i]['geo']]
    popup_text = 'Brief von: ' + senders_list[i]['sender_name'] + '<br/>An: ' + receivers_list[i]['receiver_name'] + '<br/><a href="' + links_list[i] + '">Link zur Escher Briefedition</a>'
    plugin.AntPath(path, popup=popup_text).add_to(testmap)

testmap
